In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.tensorboard import SummaryWriter
import os

In [3]:
# Set up TensorBoard log directory
log_dir = "runs/mnist_experiment"
writer = SummaryWriter(log_dir)

In [4]:
# Hyperparameters
batch_size = 64
learning_rate = 0.01
epochs = 5

In [5]:
# MNIST dataset
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

In [6]:
train_dataset = datasets.MNIST('../data', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST('../data', train=False, download=True, transform=transform)

100%|██████████| 9.91M/9.91M [00:04<00:00, 2.30MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 124kB/s]
100%|██████████| 1.65M/1.65M [00:01<00:00, 1.02MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 13.8MB/s]


In [7]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=1000, shuffle=False)

In [8]:
# Define a simple neural network
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(28 * 28, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = x.view(-1, 28 * 28)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x


model = Net()
optimizer = optim.SGD(model.parameters(), lr=learning_rate)

In [9]:
# Training loop
for epoch in range(epochs):
    model.train()
total_loss = 0
correct = 0

for batch_idx, (data, target) in enumerate(train_loader):
    optimizer.zero_grad()
    output = model(data)
    loss = F.cross_entropy(output, target)
    loss.backward()
    optimizer.step()

    total_loss += loss.item()
    pred = output.argmax(dim=1, keepdim=True)
    correct += pred.eq(target.view_as(pred)).sum().item()

avg_loss = total_loss / len(train_loader)
accuracy = correct / len(train_loader.dataset)

In [10]:
 # Log to TensorBoard
writer.add_scalar('Loss/train', avg_loss, epoch)
writer.add_scalar('Accuracy/train', accuracy, epoch)
print(f"Epoch {epoch + 1}: Loss={avg_loss:.4f}, Accuracy={accuracy:.4f}")

Epoch 5: Loss=0.5875, Accuracy=0.8481


In [16]:
# Save the trained model
model_path = "../models/mnist_model.pth"
os.makedirs(os.path.dirname(model_path), exist_ok=True)
torch.save(model.state_dict(), model_path)
print(f"Model saved to: {model_path}")

Model saved to: ../models/mnist_model.pth


In [19]:
# Evaluate the model
model = Net()
model.load_state_dict(torch.load("../models/mnist_model.pth"))
model.eval()

Net(
  (fc1): Linear(in_features=784, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=10, bias=True)
)